In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image

num_classes = 200

def load(filename):
    file = open(filename, "r") 
    image_names = file.readlines()
    images = []
    labels = []
    for name in image_names:
        label = int(name[:3])
        if label <= 2:
            im = Image.open("images/" + name.rstrip('\n'))
            H, W = im.size
            pixels = list(im.getdata())
            if not type(pixels[0]) is int:
                # todo: right now we are discarding transparent images
                image = np.array([comp for pixel in pixels for comp in pixel]).reshape(-1, H, W, 3)
                images.append(image)
                # zero-index the label
                labels.append(label - 1)
    return images, labels

images_train, labels_train = load('train.txt')
# images_test, labels_test = load('test.txt')

print(len(images_train))
print(len(labels_train))

60
60


In [15]:
# todo: use tf.contrib.layers.conv2d for 3D filter
def conv_relu(input_image, kernel_shape, bias_shape, stride = 2):
    strides = [1, stride, stride, 1]
    weights = tf.get_variable("weights", kernel_shape, initializer=tf.random_normal_initializer())
    biases = tf.get_variable("biases", bias_shape, initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input_image, weights, strides=strides, padding='SAME')
    return tf.nn.relu(conv + biases)

def conv_layers(input_image):
    # Variables created here will be named "convX/weights", "convX/biases".
    with tf.variable_scope("conv1"):
        out_channels = 15
        output = conv_relu(input_image, [5, 5, 3, out_channels], [out_channels], stride=1)
    with tf.variable_scope("conv2"):
        old_out_channels = out_channels
        out_channels = 20
        output = conv_relu(output, [5, 5, old_out_channels, out_channels], [out_channels])
    with tf.variable_scope("conv3"):
        old_out_channels = out_channels
        out_channels = 3
        return conv_relu(output, [5, 5, old_out_channels, out_channels], [out_channels])
    
# https://github.com/tensorflow/tensorflow/issues/6011
# def spp_layer(image, levels=[6, 3, 2, 1]):
#     shape = image.get_shape().as_list()
#     print(shape)
# #     if shape[1] < levels[0] ** 2 or shape[2] < levels[0] ** 2:
# #         print(shape)
# #         print('Size must be greater than {:d}x{:d}'.format(levels[0], levels[0]))
# #         return None

#     with tf.variable_scope('spp'):
#         pool_outputs = []
#         for level in levels:
#             # todo: figure out why it is surrounded by 1 (Striding over your images would mean you'd just drop whole images from training or induction.)
              # https://stackoverflow.com/questions/43790742/how-to-stride-over-multiple-channels-in-conv2d-of-tensorflow
#             window_size = [1] + [np.ceil(d / level).astype(np.int32) for d in shape[1:3]] + [1]
#             strides = [1] + [np.floor(d / level + 1).astype(np.int32) for d in shape[1:3]] + [1]
            
#             pool = tf.nn.max_pool(image, ksize=window_size, strides=strides, padding='SAME')
#             pool_outputs.append(tf.reshape(pool, [shape[0], -1]))
#         spp_pool = tf.concat(pool_outputs, axis=1)
#     return None

def max_pool_2d_nxn_regions(inputs, output_size):
    inputs_shape = tf.shape(inputs)
    h = tf.cast(tf.gather(inputs_shape, 1), tf.int32)
    w = tf.cast(tf.gather(inputs_shape, 2), tf.int32)
    
    pooling_op = tf.reduce_max
    
#         pooling_op = tf.reduce_mean

    result = []
    n = output_size
    for row in range(output_size):
        for col in range(output_size):
            # start_h = floor(row / n * h)
            start_h = tf.cast(tf.floor(tf.multiply(row / n, tf.cast(h, tf.float32))), tf.int32)
            # end_h = ceil((row + 1) / n * h)
            end_h = tf.cast(tf.ceil(tf.multiply((row + 1) / n, tf.cast(h, tf.float32))), tf.int32)
            # start_w = floor(col / n * w)
            start_w = tf.cast(tf.floor(tf.multiply(col / n, tf.cast(w, tf.float32))), tf.int32)
            # end_w = ceil((col + 1) / n * w)
            end_w = tf.cast(tf.ceil(tf.multiply((col + 1) / n, tf.cast(w, tf.float32))), tf.int32)
            pooling_region = inputs[:, start_h:end_h, start_w:end_w, :]
            pool_result = pooling_op(pooling_region, axis=(1, 2))
            result.append(pool_result)
    return result

# Modified from RikHeijdens on https://github.com/tensorflow/tensorflow/issues/6011
def spp_layer(inputs, dimensions=[3, 2, 1]):
    # todo: fix this
    # print(inputs.get_shape()[1] < tf.constant(36, dtype=tf.int32))


#     if tf.less(inputs.get_shape()[1], dimensions[0] ** 2) or tf.less(inputs.get_shape()[2], dimensions[0] ** 2):
#         print(shape)
#         print('Size must be greater than {:d}x{:d}'.format(dimensions[0], dimensions[0]))
#         return None
    pool_list = []
    for pool_dim in dimensions:
        pool_list += max_pool_2d_nxn_regions(inputs, pool_dim)
    return tf.concat(pool_list, axis=1)

# todo: might be able to move this into session
def fc_layer(image, reuse):
    return tf.contrib.layers.fully_connected(image, num_classes, activation_fn=None, scope="fc", reuse=reuse)



tf.reset_default_graph()
fc_reuse = False
with tf.variable_scope("network") as scope:
    image = tf.placeholder(tf.float32, (1,None,None,3), name='image')
    h = tf.to_float(image)
    h = conv_layers(h)
    h = spp_layer(h)
    
    if not h is None:
        h = fc_layer(h, fc_reuse)
        h = tf.reshape(h, [-1])
        accuracy = tf.reduce_mean(tf.cast(h, tf.float32))
#         logits.append(output)
#         logit_labels.append(batch_labels[index])
        fc_reuse = True

    scope.reuse_variables()
    
session = tf.Session()
session.run(tf.global_variables_initializer())

session.run([accuracy], feed_dict={image: images_train[0]})

# session.run([h], feed_dict={image: images_train[1]})

# print(len(tf.trainable_variables()))




[8330.9678]

In [14]:
tf.reset_default_graph()

epochs = 20
batch_size = 32
session = tf.Session()
session.run(tf.global_variables_initializer())

with tf.variable_scope("network") as scope:
    fc_reuse = False
    for epoch in range(epochs):
        np.random.seed(epoch)
        np.random.shuffle(images_train)
        np.random.seed(epoch)
        np.random.shuffle(labels_train)
        accuracy_vals, loss_vals = [], []

        print(len(images_train))
        for i in range(0, len(images_train) - batch_size + 1, batch_size):
            batch_images, batch_labels = images_train[i:i + batch_size], labels_train[i:i + batch_size]
            logits, logit_labels = [], []
        
            for index, image in enumerate(batch_images):
                output = tf.convert_to_tensor(image)
                output = tf.to_float(output)
                output = conv_layers(output)
                output = spp_layer(output)
                if not output is None:
                    output = fc_layer(output, fc_reuse)
                    output = tf.reshape(output, [-1])
                    logits.append(output)
                    logit_labels.append(batch_labels[index])
                    fc_reuse = True

                scope.reuse_variables()
            
            
            loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=logit_labels))
            regularization_loss = tf.losses.get_regularization_loss()
            total_loss = loss + 1e-6 * regularization_loss
            # todo: play around with optimizer
#             optimizer = tf.train.MomentumOptimizer(0.001, 0.9)
#             with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
#                 opt = optimizer.minimize(total_loss)
            correct = tf.equal(tf.argmax(logits, 1), logit_labels)
            accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
            print(session.run([correct]))
#             print(total_loss)
            
#             accuracy_val, loss_val, _ = sess.run([accuracy, total_loss, opt], feed_dict={logits: logits, logits_labels: logits_labels})
#             accuracy_vals.append(accuracy_val)
#             loss_vals.append(loss_val)

#     val_correct = []
#     for i in range(0, image_test.shape[0], batch_size):
#         batch_images, batch_labels = image_val[i:i + batch_size], label_val[i:i + batch_size]
#         val_correct.extend( sess.run(correct, feed_dict={inputs: batch_images, labels: batch_labels}) )
#     print('[%3d] Accuracy: %0.3f  \t  Loss: %0.3f  \t  validation accuracy: %0.3f'%(epoch, np.mean(accuracy_vals), np.mean(loss_vals), np.mean(val_correct)))

60
Tensor("network/Mean:0", shape=(), dtype=float32)


InvalidArgumentError: Current implementation does not yet support strides in the batch and depth dimensions.
	 [[Node: network/conv2/Conv2D = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[2, 2, 2, 2], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/cpu:0"](network/conv1/Relu, network/conv2/weights/read)]]

Caused by op 'network/conv2/Conv2D', defined at:
  File "/Users/michaelscaria/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/michaelscaria/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/michaelscaria/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/michaelscaria/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/michaelscaria/anaconda/lib/python3.6/site-packages/ipykernel/kernelapp.py

In [ ]:
# https://github.com/tensorflow/tensorflow/issues/6011
# def spp_layer(image, levels=[6, 3, 2, 1]):
#     shape = image.get_shape().as_list()
#     print(shape)
#     if shape[1] < levels[0] ** 2 or shape[2] < levels[0] ** 2:
#         print(shape)
#         print('Size must be greater than {:d}x{:d}'.format(levels[0], levels[0]))
#         return None

#     with tf.variable_scope('spp'):
#         pool_outputs = []
#         for level in levels:
#             # todo: figure out why it is surrounded by 1
# #             window_size = [1] + [tf.cast(tf.ceil(d / level), tf.int32) for d in tf.shape(image)[1:3]] + [1]
# #             strides = [1] + [tf.cast(tf.floor(d / level + 1), tf.int32) for d in tf.shape(image)[1:3]] + [1]
# #             win_size = [1, 
# #                         tf.cast(tf.ceil(tf.shape(image)[1]  / level), tf.int32), 
# #                         tf.cast(tf.ceil(tf.shape(image)[2]  / level), tf.int32), 
# #                         1]
# #             strides = [1, 
# #                         tf.cast(tf.floor(tf.shape(image)[1] / level + 1), tf.int32), 
# #                         tf.cast(tf.floor(tf.shape(image)[2] / level + 1), tf.int32), 
# #                         1]
# #             print(tf.Session().run([image.get_shape()[1]]))
#             b = tf.shape(image)[1]
#             print(b)
#             print(b.eval(tf.Session()))
#             window_size = np.ceil(b / level).astype(np.int32)
#             print(window_size)
            
# #             pool = tf.nn.max_pool(image, ksize=window_size, strides=strides, padding='SAME')
# #             pool_outputs.append(tf.reshape(pool, [shape[0], -1]))
# #         spp_pool = tf.concat(pool_outputs, axis=1)
#     return None

# def spp_layer(input_, levels=[3, 2, 1], name = 'SPP_layer'):
#     '''Multiple Level SPP layer.
#        Works for levels=[1, 2, 3, 6].'''
    
#     shape = input_.get_shape().as_list()
#     print(shape)
#     with tf.variable_scope(name):
#         pool_outputs = []
#         for l in levels:
#             p = [1, tf.cast(tf.ceil(tf.shape(input_)[1]  / l), tf.int32), tf.cast(tf.ceil(tf.shape(input_)[2]  / l), tf.int32), 1]
#             print(p)
#             pool = tf.nn.max_pool(input_, ksize=p, 
#                                       strides=[1, tf.cast(tf.floor(tf.shape(input_)[1]  / l + 1), tf.int32), tf.floor(tf.shape(input_)[2] / l + 1), 1], 
#                                       padding='SAME')
# #             print "Pool Level {:}: shape {:}".format(l, pool.get_shape().as_list())
#             pool_outputs.append(tf.reshape(pool, [tf.shape(input_)[0], -1]))
#         spp_pool = tf.concat(pool_outputs, axis=1)
#     return spp_pool


